In [0]:
!pip install torch torchvision pytorch-transformers
!pip install scikit-learn pandas tensorflow lime

from __future__ import absolute_import, division, print_function
import torch

torch.manual_seed(0)
import csv
import logging
import os
import sys
from io import open

from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import matthews_corrcoef, f1_score

from multiprocessing import Pool, cpu_count
from tqdm import tqdm

logger = logging.getLogger(__name__)
csv.field_size_limit(2147483647)
# writerobj = SummaryWriter()

class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                if sys.version_info[0] == 2:
                    line = list(unicode(cell, 'utf-8') for cell in line)
                lines.append(line)
            return lines


class BinaryProcessor(DataProcessor):
    """Processor for the binary data sets"""

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir,data_type="dev"):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, data_type+".tsv")), "dev")

    def get_labels(self):
        """See base class."""
        return ["0","1","2","3"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines):
            
            guid = "%s-%s" % (set_type, i)
            try:
                text_a = line[3]
                label = line[1]
            except:
                print(line)
            #print(text_a,label)
            examples.append(
                InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        return examples


def convert_example_to_feature(example_row, pad_token=0,
sequence_a_segment_id=0, sequence_b_segment_id=1,
cls_token_segment_id=1, pad_token_segment_id=0,
mask_padding_with_zero=True):
    
   
    
    example, label_map, max_seq_length, tokenizer, output_mode, cls_token_at_end, cls_token, sep_token, cls_token_segment_id, pad_on_left, pad_token_segment_id = example_row
    #print(example.text_a,"label-------->",example.label)
    tokens_a = tokenizer.tokenize(example.text_a)

    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[:(max_seq_length - 2)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids:   0   0  0    0    0     0       0   0   1  1  1  1   1   1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids:   0   0   0   0  0     0   0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = tokens_a + [sep_token]
    segment_ids = [sequence_a_segment_id] * len(tokens)

    if tokens_b:
        tokens += tokens_b + [sep_token]
        segment_ids += [sequence_b_segment_id] * (len(tokens_b) + 1)

    if cls_token_at_end:
        tokens = tokens + [cls_token]
        segment_ids = segment_ids + [cls_token_segment_id]
    else:
        tokens = [cls_token] + tokens
        segment_ids = [cls_token_segment_id] + segment_ids

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

    # Zero-pad up to the sequence length.
    padding_length = max_seq_length - len(input_ids)
    if pad_on_left:
        input_ids = ([pad_token] * padding_length) + input_ids
        input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
        segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
    else:
        input_ids = input_ids + ([pad_token] * padding_length)
        input_mask = input_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
        segment_ids = segment_ids + ([pad_token_segment_id] * padding_length)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length
    
    #print(label_map)

    if output_mode == "classification":
        label_id = label_map[example.label]
    elif output_mode == "regression":
        label_id = float(example.label)
    else:
        raise KeyError(output_mode)

    return InputFeatures(input_ids=input_ids,
                        input_mask=input_mask,
                        segment_ids=segment_ids,
                        label_id=label_id)
    

def convert_examples_to_features(examples, label_list, max_seq_length,
                                 tokenizer, output_mode,
                                 cls_token_at_end=False, pad_on_left=False,
                                 cls_token='[CLS]', sep_token='[SEP]', pad_token=0,
                                 sequence_a_segment_id=0, sequence_b_segment_id=1,
                                 cls_token_segment_id=1, pad_token_segment_id=0,
                                 mask_padding_with_zero=True,
                                 process_count=cpu_count() - 2):
    """ Loads a data file into a list of `InputBatch`s
        `cls_token_at_end` define the location of the CLS token:
            - False (Default, BERT/XLM pattern): [CLS] + A + [SEP] + B + [SEP]
            - True (XLNet/GPT pattern): A + [SEP] + B + [SEP] + [CLS]
        `cls_token_segment_id` define the segment id associated to the CLS token (0 for BERT, 2 for XLNet)
    """

    label_map = {label : i for i, label in enumerate(label_list)}
    #print(label_map)

    examples = [(example, label_map, max_seq_length, tokenizer, output_mode, cls_token_at_end, cls_token, sep_token, cls_token_segment_id, pad_on_left, pad_token_segment_id) for example in examples]

    with Pool(process_count) as p:
        features = list(tqdm(p.imap(convert_example_to_feature, examples, chunksize=100), total=len(examples)))

    return features


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


processors = {
    "binary": BinaryProcessor
}

output_modes = {
    "binary": "classification"
}

GLUE_TASKS_NUM_LABELS = {
    "binary": 4
}


     |████████████████████████████████| 184kB 3.4MB/s 
     |████████████████████████████████| 1.0MB 27.2MB/s 
     |████████████████████████████████| 870kB 34.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=df81d73ceaa35461e66889744220f02a63753daeff87347b782d6452ee376380
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
     |████████████████████████████████| 276kB 3.3MB/s 
  Created wheel for lime: filename=lime-0.1.1.37-cp36-none-any.whl size=284277 sha256=c9e75f44ad68f5d25cabe1541713cdd60d58a62c7775fdbc0e618970f65ce0ee
  Stored in directory: /root/.cache/pip/wheels/c1/38/e7/50d75d4fb75afa604570dc42f20c5c5f5ab26d3fbe8d6ef27b
  Created wheel for progressbar: filename=progressbar-2.5-cp36-none-any.whl size=12074 sha256=9a5a4c62ed3b4c445eec44f931f12538e825113ff2721215d9d5fd73e03628fa
  Stored in directory: /root/.cache/pip/wheels/c0/e9/6b/ea01090

# Processing

In [0]:
import pandas as pd
from tqdm import tqdm_notebook
import io
from google.colab import files
from sklearn.model_selection import train_test_split

# trainfile = files.upload()
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train_df = pd.read_csv("/content/drive/My Drive/exp1/train1.tsv",delimiter="\t")
test_df = pd.read_csv("/content/drive/My Drive/exp1/test1.tsv",delimiter="\t")
dev_df = test_df

In [0]:
train_df

,text,label
0,This version of the agent was able to perform ...,2
1,"For a 2*2 matrix, the agent initially checks w...",1
2,The time complexity remains the same as previo...,2
3,"At the same time, there is no inductivelearnin...",2
4,"Thereafter, generalizing algorithms by avoidin...",1
...,...,...
271,"To beginwith, from BP-2, it can be seen that i...",2
272,Analysisof problems reveal that rule ATowardsI...,3
273,"Problems CP-4 was solved, withoutany other err...",2
274,"Based on analysis from submission8, for this s...",1


In [0]:
train_df = pd.DataFrame({
    'id':range(len(train_df)),
    'label':train_df["label"].astype(int)-1,
    'alpha':['a']*train_df.shape[0],
    'text': train_df["text"].replace(r'\n', ' ', regex=True)
})

train_df.head()

,id,label,alpha,text
0,0,1,a,This version of the agent was able to perform ...
1,1,0,a,"For a 2*2 matrix, the agent initially checks w..."
2,2,1,a,The time complexity remains the same as previo...
3,3,1,a,"At the same time, there is no inductivelearnin..."
4,4,0,a,"Thereafter, generalizing algorithms by avoidin..."


In [0]:
dev_df = pd.DataFrame({
    'id':range(len(dev_df)),
    'label':dev_df["label"].astype(int)-1,
    'alpha':['a']*dev_df.shape[0],
    'text': dev_df["text"].replace(r'\n', ' ', regex=True)
})

dev_df.head()

,id,label,alpha,text
0,0,1,a,I feel that my agent is similar to me in some ...
1,1,1,a,This possibly indicates that the update I made...
2,2,0,a,This checks if the difference in black pixels ...
3,3,0,a,It now computes thecombination of two images u...
4,4,1,a,Most importantly the performance on test set a...


In [0]:
test_df = pd.DataFrame({
    'id':range(len(test_df)),
     'label':test_df["label"].astype(int)-1,
    'alpha':['a']*test_df.shape[0],
    'text': test_df["text"].replace(r'\n', ' ', regex=True)
})

test_df.head()

,id,label,alpha,text
0,0,1,a,I feel that my agent is similar to me in some ...
1,1,1,a,This possibly indicates that the update I made...
2,2,0,a,This checks if the difference in black pixels ...
3,3,0,a,It now computes thecombination of two images u...
4,4,1,a,Most importantly the performance on test set a...


In [0]:
!mkdir data_new_1
train_df.to_csv('data_new_1/train.tsv', sep='\t', index=False, header=False, columns=train_df.columns)
dev_df.to_csv('data_new_1/dev.tsv', sep='\t', index=False, header=False, columns=dev_df.columns)
test_df.to_csv('data_new_1/test.tsv', sep='\t', index=False, header=False, columns=test_df.columns)

In [0]:
!pip install pytorch_transformers


In [0]:
from __future__ import absolute_import, division, print_function

import glob
import logging
import os
import random
import json

import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
import random
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm_notebook, trange


from pytorch_transformers import (WEIGHTS_NAME, BertConfig, BertForSequenceClassification, BertTokenizer,
                                  XLMConfig, XLMForSequenceClassification, XLMTokenizer, 
                                  XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer,
                                  RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)

from pytorch_transformers import AdamW, WarmupLinearSchedule

# from utils import (convert_examples_to_features,
#                         output_modes, processors)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [0]:
args_roberta = {
    'data_dir': 'data_new_1/',
    'model_type':  'roberta',
    'model_name': 'roberta-base',
    'task_name': 'binary',
    'output_dir': 'outputs_robert-base-0.9_taskbc',
    'cache_dir': 'cache_roberta/',
    'do_train': True,
    'do_eval': True,
    'fp16': False,
    'fp16_opt_level': 'O1',
    'max_seq_length': 128,
    'output_mode': 'classification',
    'train_batch_size': 16,
    'eval_batch_size': 1,

    'gradient_accumulation_steps': 1,
    'num_train_epochs': 10,
    'weight_decay': 0,
    'learning_rate': 4e-8,
    'adam_epsilon': 1e-8,
    'warmup_steps': 1000,
    'max_grad_norm': 1.0,

    'logging_steps': 50,
    'evaluate_during_training': False,
    'save_steps': 5000,
    'eval_all_checkpoints': True,

    'overwrite_output_dir': False,
    'reprocess_input_data': False,
    'notes': 'Using Yelp Reviews dataset'
}
args_bert=args_roberta
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
with open('args_bert.json', 'w') as f:
    json.dump(args_bert, f)
with open('args_roberta.json', 'w') as f:
    json.dump(args_roberta, f)

In [0]:
if os.path.exists(args_bert['output_dir']) and os.listdir(args_bert['output_dir']) and args_bert['do_train'] and not args_bert['overwrite_output_dir']:
    raise ValueError("Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(args_bert['output_dir']))

#Create roberta directory
if os.path.exists(args_roberta['output_dir']) and os.listdir(args_roberta['output_dir']) and args_roberta['do_train'] and not args_roberta['overwrite_output_dir']:
    raise ValueError("Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(args_roberta['output_dir']))

In [0]:
MODEL_CLASSES = {
    'bert': (BertConfig, BertForSequenceClassification, BertTokenizer),
    'xlnet': (XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer),
    'xlm': (XLMConfig, XLMForSequenceClassification, XLMTokenizer),
    'roberta': (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)
}

config_class_bert, model_class_bert, tokenizer_class_bert = MODEL_CLASSES[args_bert['model_type']]

MODEL_CLASSES = {
    'bert': (BertConfig, BertForSequenceClassification, BertTokenizer),
    'xlnet': (XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer),
    'xlm': (XLMConfig, XLMForSequenceClassification, XLMTokenizer),
    'roberta': (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)
}

#roberta
config_class_roberta, model_class_roberta, tokenizer_class_roberta = MODEL_CLASSES[args_roberta['model_type']]


In [0]:
# config_bert = config_class_bert.from_pretrained(args_bert['model_name'], num_labels=4, finetuning_task=args_bert['task_name'])#, output_hidden_states=True)
# tokenizer_bert = tokenizer_class_bert.from_pretrained(args_bert['model_name'])

config_bert=config_roberta = config_class_roberta.from_pretrained(args_roberta['model_name'], num_labels=4, finetuning_task=args_roberta['task_name'])
tokenizer_bert=tokenizer_roberta = tokenizer_class_roberta.from_pretrained(args_roberta['model_name'])
model_bert=model_roberta= model_class_roberta.from_pretrained(args_roberta['model_name'],num_labels=4)
model_bert.to(device);

INFO:pytorch_transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json not found in cache or force_download set to True, downloading to /tmp/tmpahqs4dax
100%|██████████| 524/524 [00:00<00:00, 183917.60B/s]
INFO:pytorch_transformers.file_utils:copying /tmp/tmpahqs4dax to cache at /root/.cache/torch/pytorch_transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.a7ab0e5de2d8321d6d6a15b199110f2c99be72976b7d151423cb8d8c261a13b6
INFO:pytorch_transformers.file_utils:creating metadata file for /root/.cache/torch/pytorch_transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.a7ab0e5de2d8321d6d6a15b199110f2c99be72976b7d151423cb8d8c261a13b6
INFO:pytorch_transformers.file_utils:removing temp file /tmp/tmpahqs4dax
INFO:pytorch_transformers.modeling_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at /root/.cache/torch/pytorch_transformers/e

In [0]:
model_bert = model_class_roberta.from_pretrained(args_bert['model_name'],num_labels=4)
# for name, param in model_bert.named_parameters():
# 	if 'classifier' not in name: # classifier layer
# 		param.requires_grad = False

INFO:pytorch_transformers.modeling_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at /root/.cache/torch/pytorch_transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.a7ab0e5de2d8321d6d6a15b199110f2c99be72976b7d151423cb8d8c261a13b6
INFO:pytorch_transformers.modeling_utils:Model config {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 4,
  "output_attentions": false,
  "output_hidden_states": false,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 1,
  "vocab_size": 50265
}

INFO:pytorch_transformers.modeling_utils:loading weights file https://

In [0]:

model_bert.to(device);


In [0]:
device

device(type='cuda', index=0)

In [0]:
task = args_bert['task_name']

processor = processors[task]()
label_list = processor.get_labels()
num_labels = len(label_list)

In [0]:
def load_and_cache_examples(task, tokenizer,argus, evaluate=False,undersample_scale_factor=1,data_type="dev"):
    args=argus.copy()
    processor = processors[task]()
    output_mode = args['output_mode']
    
    mode = 'dev' if evaluate else 'train'
    cached_features_file = os.path.join(args['data_dir'], "cached_"+str(mode)+"_"+str(args['model_name'])+"_"+str(args['max_seq_length'])+"_"+str(task))
    
    if os.path.exists(cached_features_file) and not args['reprocess_input_data'] and False:
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
               
    else:
        logger.info("Creating features from dataset file at %s", args['data_dir'])
        label_list = processor.get_labels()
        examples = processor.get_dev_examples(args['data_dir'],data_type) if evaluate else processor.get_train_examples(args['data_dir'])
#         print(len(examples))
        examples  = [example for example in examples]
        #print((examples))
        
        features = convert_examples_to_features(examples, label_list, args['max_seq_length'], tokenizer, output_mode,
            cls_token_at_end=bool(args['model_type'] in ['xlnet']),            # xlnet has a cls token at the end
            cls_token=tokenizer.cls_token,
            sep_token=tokenizer.sep_token,
            cls_token_segment_id=2 if args['model_type'] in ['xlnet'] else 0,
            pad_on_left=bool(args['model_type'] in ['xlnet']),                 # pad on the left for xlnet
            pad_token_segment_id=4 if args['model_type'] in ['xlnet'] else 0,
            process_count=2)
        
        logger.info("Saving features into cached file %s", cached_features_file)
        torch.save(features, cached_features_file)
        
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    if output_mode == "classification":
        all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)
    elif output_mode == "regression":
        all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.float)

    dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    return dataset

In [0]:
def train(train_dataset, model, tokenizer,argus):
    global writerobj
    args=argus.copy()
    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args['train_batch_size'])
    
    t_total = len(train_dataloader) // args['gradient_accumulation_steps'] * args['num_train_epochs']
    
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args['weight_decay']},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args['learning_rate'], eps=args['adam_epsilon'])
    scheduler = WarmupLinearSchedule(optimizer, warmup_steps=args['warmup_steps'], t_total=t_total)
    
    if args['fp16']:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args['fp16_opt_level'])
        
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args['num_train_epochs'])
    logger.info("  Total train batch size  = %d", args['train_batch_size'])
    logger.info("  Gradient Accumulation steps = %d", args['gradient_accumulation_steps'])
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    train_iterator = trange(int(args['num_train_epochs']), desc="Epoch")
    
    for _ in train_iterator:
        epoch_iterator = tqdm_notebook(train_dataloader, desc="Iteration")
        for step, batch in enumerate(epoch_iterator):
            model.train()
            batch = tuple(t.to(device) for t in batch)
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2] if args['model_type'] in ['bert', 'xlnet'] else None,  # XLM don't use segment_ids
                      'labels':         batch[3]}
            outputs = model(**inputs)
            #print(model.bert(**inputs).shape)
            loss = outputs[0]  # model outputs are always tuple in pytorch-transformers (see doc)
            print("\r%f" % loss, end='')
            #writerobj.add_scalar('Loss/train', float(loss), global_step)
            

            if args['gradient_accumulation_steps'] > 1:
                loss = loss / args['gradient_accumulation_steps']
                
               
                
 
            if args['fp16']:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
                torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args['max_grad_norm'])
                
            else:
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), args['max_grad_norm'])
            
            tr_loss += loss.item()
            if (step + 1) % args['gradient_accumulation_steps'] == 0:
                
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args['logging_steps'] > 0 and global_step % args['logging_steps'] == 0:
                    # Log metrics
                    if args['evaluate_during_training']:  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(model, tokenizer)
                        

                    logging_loss = tr_loss
                    

                if args['save_steps'] > 0 and global_step % args['save_steps'] == 0:
                    # Save model checkpoint
                    output_dir = os.path.join(args['output_dir'], 'checkpoint')
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    logger.info("Saving model checkpoint to %s", output_dir)
            

    return global_step, tr_loss / global_step

In [0]:
def evaluate_runtime(data,gt,tokenizer,prefix=""):
    eval_dataset = load_and_cache_examples_runtime(data,gt, tokenizer, undersample_scale_factor=1, evaluate=True)

    args=args_bert
    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args['eval_batch_size'])

    print("Run time evaluation on progress!!")
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None
    probs=None
    net_input=None
    featurelis=[]
    for batch in tqdm_notebook(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2] if args['model_type'] in ['bert', 'xlnet'] else None,  # XLM don't use segment_ids
                      'labels':         batch[3]}
            outputs = model(**inputs)
            tmp_eval_loss, logits = outputs[:2]
            tmp_eval_loss2, logits2 = outputs[:2]
            

            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        
        if preds is None:
            preds = logits.detach().cpu().numpy()
            probs= torch.nn.functional.softmax(logits2.detach().cpu(), dim=1)
            out_label_ids = inputs['labels'].detach().cpu().numpy()
            net_input=inputs['input_ids'].detach().cpu().numpy()
            featurelis.append(model.bert(inputs["input_ids"],inputs["token_type_ids"],inputs["attention_mask"])[1].data.cpu().numpy())
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)
            probs=np.append(probs,torch.nn.functional.softmax(logits2.detach().cpu(), dim=1), axis=0)
            net_input=np.append(net_input,inputs['input_ids'].detach().cpu().numpy(),axis=0)
            featurelis.append(model.bert(inputs["input_ids"],inputs["token_type_ids"],inputs["attention_mask"])[1].data.cpu().numpy())
              

    return probs,preds,featurelis#,out_label_ids,net_input

In [0]:
from sklearn.metrics import mean_squared_error, matthews_corrcoef, confusion_matrix, classification_report
from scipy.stats import pearsonr

def get_mismatched(labels, preds):
    mismatched = labels != preds
    examples = processor.get_dev_examples(args_bert['data_dir'])
    wrong = [i for (i, v) in zip(examples, mismatched) if v]
    
    return wrong

def get_eval_report(labels, preds):
    mcc = matthews_corrcoef(labels, preds)
    print(classification_report(labels, preds))
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    print(classification_report(labels, preds))
   
    return {
        "mcc": mcc,
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn
    }, get_mismatched(labels, preds)

def compute_metrics(task_name, preds, labels):
    assert len(preds) == len(labels)
    return get_eval_report(labels, preds)

def evaluate(model, argus, tokenizer, prefix="",datatype="dev"):
    
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    args=argus.copy()
    eval_output_dir = args['output_dir']

    results = {}
    EVAL_TASK = args['task_name']

    eval_dataset = load_and_cache_examples(EVAL_TASK, tokenizer, args, undersample_scale_factor=1, evaluate=True,data_type=datatype)
    if not os.path.exists(eval_output_dir):
        os.makedirs(eval_output_dir)


    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args['eval_batch_size'])

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    #print(len(eval_dataset))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args['eval_batch_size'])
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None
    probs=None
    net_input=None
    for batch in tqdm_notebook(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2] if args['model_type'] in ['bert', 'xlnet'] else None,  # XLM don't use segment_ids
                      'labels':         batch[3]}
            outputs = model(**inputs)
            feature=model.bert(inputs["input_ids"],inputs["token_type_ids"],inputs["attention_mask"])[1].cpu().numpy()
            print(feature.shape)
            tmp_eval_loss, logits = outputs[:2]

            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        
        if preds is None:
            preds = logits.detach().cpu().numpy()
            probs= preds.copy()
            out_label_ids = inputs['labels'].detach().cpu().numpy()
            net_input=inputs['input_ids'].detach().cpu().numpy()
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)
            probs=np.append(probs,logits.detach().cpu().numpy(), axis=0)
            net_input=np.append(net_input,inputs['input_ids'].detach().cpu().numpy(),axis=0)
            
    eval_loss = eval_loss / nb_eval_steps
    if args['output_mode'] == "classification":
        preds = np.argmax(preds, axis=1)
    elif args['output_mode'] == "regression":
        preds = np.squeeze(preds)
    result, wrong = compute_metrics(EVAL_TASK, preds, out_label_ids)
    results.update(result)

    output_eval_file = os.path.join(eval_output_dir, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return results, wrong,probs,preds,out_label_ids,net_input


def extract_features(model, argus, tokenizer, prefix="",datatype="dev"):
    
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    args=argus.copy()
    eval_output_dir = args['output_dir']

    results = {}
    EVAL_TASK = args['task_name']

    eval_dataset = load_and_cache_examples(EVAL_TASK, tokenizer, args, undersample_scale_factor=1, evaluate=True,data_type=datatype)
    if not os.path.exists(eval_output_dir):
        os.makedirs(eval_output_dir)


    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args['eval_batch_size'])

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    #print(len(eval_dataset))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args['eval_batch_size'])


    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None
    probs=None
    net_input=None
    
    #save data
    all_features=[]
    all_sentences=[]
    all_labels=[]

    for batch in tqdm_notebook(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(device) for t in batch)
        # print(tokenizer.decode(batch[0])#.decode)
        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2] if args['model_type'] in ['bert', 'xlnet'] else None,  # XLM don't use segment_ids
                      'labels':         batch[3]}
            outputs = model(**inputs)
            feature=model.roberta(inputs["input_ids"],inputs["token_type_ids"],inputs["attention_mask"])[1].cpu().numpy()
            
            all_features.append(feature[0])
            all_sentences.append(feature)
            all_labels.append(batch[3].cpu().numpy())

    return all_features,all_sentences,all_labels

In [0]:
#Train_bert
if args_bert['do_train'] and False:
    print("Training Bert!!")
    train_dataset = load_and_cache_examples(task, tokenizer_bert,args_bert,undersample_scale_factor=1)
    #print(train_dataset)
    global_step, tr_loss = train(train_dataset, model_bert, tokenizer_bert,args_bert)
    logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

In [0]:
#Save bert
if args_bert['do_train']:
    if not os.path.exists(args_bert['output_dir']):
            os.makedirs(args_bert['output_dir'])
    logger.info("Saving model checkpoint to %s", args_bert['output_dir'])
    
    model_to_save = model_bert.module if hasattr(model_bert, 'module') else model_bert  # Take care of distributed/parallel training
    model_to_save.save_pretrained(args_bert['output_dir'])
    tokenizer_bert.save_pretrained(args_bert['output_dir'])
    torch.save(args_bert, os.path.join(args_bert['output_dir'], 'training_args.bin'))


INFO:__main__:Saving model checkpoint to outputs_robert-base-0.9_taskbc


# Get test set predictions and probs for BERT

In [0]:
results = {}
if args_roberta['do_eval']:
    checkpoints = [args_roberta['output_dir']]
    if args_roberta['eval_all_checkpoints']:
        checkpoints = list(os.path.dirname(c) for c in sorted(glob.glob(args_roberta['output_dir']+ '/**/' + WEIGHTS_NAME, recursive=True)))
        logging.getLogger("pytorch_transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
    logger.info("Evaluate the following checkpoints: %s", checkpoints)
    for checkpoint in checkpoints:
        print(checkpoint)
        if True:
            global_step = checkpoint.split('-')[-1] if len(checkpoints) > 1 else ""
            model = model_class_roberta.from_pretrained(checkpoint)
            model.to(device)
            xtest, sentence,ytest= extract_features(model, args_roberta,tokenizer_roberta, prefix=global_step,datatype="test")


results = {}
if args_roberta['do_eval']:
    checkpoints = [args_roberta['output_dir']]
    if args_roberta['eval_all_checkpoints']:
        checkpoints = list(os.path.dirname(c) for c in sorted(glob.glob(args_roberta['output_dir']+ '/**/' + WEIGHTS_NAME, recursive=True)))
        logging.getLogger("pytorch_transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
    logger.info("Evaluate the following checkpoints: %s", checkpoints)
    for checkpoint in checkpoints:
        print(checkpoint)
        if True:
            global_step = checkpoint.split('-')[-1] if len(checkpoints) > 1 else ""
            model = model_class_roberta.from_pretrained(checkpoint)
            model.to(device)
            xtrain, sentence,ytrain= extract_features(model, args_roberta,tokenizer_roberta, prefix=global_step,datatype="train")


INFO:__main__:Evaluate the following checkpoints: ['outputs_robert-base-0.9_taskbc']


outputs_robert-base-0.9_taskbc


INFO:__main__:Creating features from dataset file at data_new_1/
100%|██████████| 184/184 [00:00<00:00, 614.36it/s]
INFO:__main__:Saving features into cached file data_new_1/cached_dev_roberta-base_128_binary
INFO:__main__:***** Running evaluation  *****
INFO:__main__:  Num examples = 184
INFO:__main__:  Batch size = 1


INFO:__main__:Evaluate the following checkpoints: ['outputs_robert-base-0.9_taskbc']



outputs_robert-base-0.9_taskbc


INFO:__main__:Creating features from dataset file at data_new_1/
100%|██████████| 276/276 [00:00<00:00, 812.34it/s]
INFO:__main__:Saving features into cached file data_new_1/cached_dev_roberta-base_128_binary
INFO:__main__:***** Running evaluation  *****
INFO:__main__:  Num examples = 276
INFO:__main__:  Batch size = 1


In [0]:

#Runtime execution based ideas
def load_and_cache_examples_runtime(data,gt,tokenizer, evaluate=False, undersample_scale_factor=1,task="binary"):
    processor = processors[task]()
    output_mode = args_bert['output_mode']
    args=args_bert
    
    examples_new = []
    count=0
    for  line,lab in zip(data,gt):
            guid = "%s-%s" % ("test", count)
            text_a = line
            label = lab
            #print(text_a,label)
            examples_new.append(
                InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
            count+=1
    
    if True:
        logger.info("Creating features from dataset file at %s", args['data_dir'])
        label_list = processor.get_labels()
        examples = examples_new#processor.get_dev_examples(args['data_dir']) if evaluate else processor.get_train_examples(args['data_dir'])
#         print(len(examples))
        examples  = [example for example in examples]
        #print((examples))
        
        features = convert_examples_to_features(examples, label_list, args['max_seq_length'], tokenizer, output_mode,
            cls_token_at_end=bool(args['model_type'] in ['xlnet']),            # xlnet has a cls token at the end
            cls_token=tokenizer.cls_token,
            sep_token=tokenizer.sep_token,
            cls_token_segment_id=2 if args['model_type'] in ['xlnet'] else 0,
            pad_on_left=bool(args['model_type'] in ['xlnet']),                 # pad on the left for xlnet
            pad_token_segment_id=4 if args['model_type'] in ['xlnet'] else 0,
            process_count=2)
        
       
        
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    if output_mode == "classification":
        all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)
    elif output_mode == "regression":
        all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.float)

    dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    return dataset

In [0]:
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.utils import class_weight
from sklearn.model_selection import RandomizedSearchCV,StratifiedKFold
from sklearn.dummy import DummyClassifier
import pickle


params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

ytrain=np.array(ytrain).ravel()
ytest=np.array(ytest).ravel()


class_weights = list(class_weight.compute_class_weight('balanced',np.unique(ytrain),np.array(ytrain)))
w_array = np.ones(ytrain.shape[0], dtype = 'float')

for i, val in enumerate(ytrain):
    w_array[i] = class_weights[val-1]

clf= SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=100, random_state=42,class_weight="balanced",warm_start=True)
clf2=XGBClassifier(n_estimators=200,random_state=10,max_depth=3,learning_rate =0.1)

clf.fit(np.array(xtrain),np.array(ytrain).astype(np.int32))
clf2.fit(np.array(xtrain),np.array(ytrain).astype(np.int32))

ypred=clf.predict(xtest)
ypred2=clf2.predict(xtest)

from sklearn.metrics import classification_report

print("SGD")
print(classification_report(np.array(ytest).ravel(),ypred,digits=5))


print("XGB")
print(classification_report(np.array(ytest).ravel(),ypred2,digits=5))

if True:
  files = open('xtrain.pkl', 'wb')

  # dump information to that file
  pickle.dump(xtrain, files)
  files = open('ytrain.pkl', 'wb')

  # dump information to that file
  pickle.dump(ytrain, files)

  files = open('xtest.pkl', 'wb')

  # dump information to that file
  pickle.dump(xtest, files)

  files = open('ytest.pkl', 'wb')

  # dump information to that file
  pickle.dump(ytest, files)
  files.close()
  print(np.array(xtrain).shape)

#change classifier here
clfimp=clf2

SGD
              precision    recall  f1-score   support

           0    0.00000   0.00000   0.00000        58
           1    0.00000   0.00000   0.00000        90
           2    0.00000   0.00000   0.00000        12
           3    0.13043   1.00000   0.23077        24

    accuracy                        0.13043       184
   macro avg    0.03261   0.25000   0.05769       184
weighted avg    0.01701   0.13043   0.03010       184

XGB
              precision    recall  f1-score   support

           0    0.73333   0.37931   0.50000        58
           1    0.55944   0.88889   0.68670        90
           2    0.25000   0.08333   0.12500        12
           3    0.42857   0.12500   0.19355        24

    accuracy                        0.57609       184
   macro avg    0.49284   0.36913   0.37631       184
weighted avg    0.57700   0.57609   0.52689       184

(276, 768)


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
!mkdir set2_bert_inter

mkdir: cannot create directory ‘set2_bert_inter’: File exists


BERT LIME INTERPRETABILITY TEST

In [0]:
def get_probs_fts(features_bert):
    global clfimp
    return clfimp.predict_proba(np.array(features_bert))

def callme(a,b=None):
    proboab,preds,net_inputs=evaluate_runtime(a,[str(1) for i in a],tokenizer_bert,prefix="")
    net_inputs=np.array(net_inputs)
    net_inputs=net_inputs.squeeze()
    print(net_inputs.shape)
    probs_new=clfimp.predict_proba(net_inputs)
    return probs_new
            
    
from lime import lime_text
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=["0","1","2","3"])
index=0
for txt in test_df["text"]:
    print(txt)
    exp = explainer.explain_instance(txt, callme, num_features=128,top_labels=4)
    #exp.show_in_notebook(text=True)
    exp.save_to_file("set2_bert_inter/"+"results_"+str(index)+".html")
    index+=1


(5000, 768)
This agent still can’t handle problems involving shapes which have shading.


/usr/local/lib/python3.6/dist-packages/lime/lime_text.py:116: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
INFO:__main__:Creating features from dataset file at data_new_1/
100%|██████████| 5000/5000 [00:01<00:00, 2522.80it/s]


Run time evaluation on progress!!



(5000, 768)
This agent revision has improved performance across all E sets, as this union algorithm is both simple and fairly general.


/usr/local/lib/python3.6/dist-packages/lime/lime_text.py:116: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
INFO:__main__:Creating features from dataset file at data_new_1/
100%|██████████| 5000/5000 [00:02<00:00, 2401.19it/s]


...

Run time evaluation on progress!!



(5000, 768)
Most importantly the performance on test set and ravens setsimproved to 6/12 and the developed rule also solved BP-12


/usr/local/lib/python3.6/dist-packages/lime/lime_text.py:116: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
INFO:__main__:Creating features from dataset file at data_new_1/
100%|██████████| 5000/5000 [00:02<00:00, 2446.66it/s]


Run time evaluation on progress!!


Buffered data was truncated after reaching the output size limit.